This is the third experiment where the model is given all the 98 focal slices to be trained for. The model is also saved for quick use.

In [1]:
autofocus_path='/mnt/Data/Autofocus/'

autofocus_train_path=autofocus_path+"Train/"
autofocus_test_path=autofocus_path+"Test/"
autofocus_cache_path=autofocus_path+"Cache/"
autofocus_exp_path=autofocus_path+"Exp3/"

In [2]:
import os
import numpy as np
import shutil
import cv2
import time
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

print(tf.version.VERSION)

2.16.2


Load the dataset from the Cache folder with only read permision

In [4]:
def load_variable_from_file(filename):
    try:
        with open(filename, 'r') as file:
            variable = file.read()
        print(f"Variable loaded successfully from {filename}")
        return variable
    except Exception as e:
        print(f"Error occurred while loading variable from {filename}: {e}")
        return None
    
_dataset_len=int(load_variable_from_file(autofocus_cache_path+"length.dat"))

shutil.copy(autofocus_cache_path+"dataset.dat",autofocus_exp_path+"dataset.dat")
shutil.copy(autofocus_cache_path+"patch.dat",autofocus_exp_path+"patch.dat")

dataset=np.memmap(autofocus_exp_path+"dataset.dat", dtype='int8', mode='r', shape=(_dataset_len,128,128,98))


Variable loaded successfully from /mnt/Data/Autofocus/Cache/length.dat


Incase the dataset is too large, this function effectively uses every kind of image scene with a reduced patch size making the dataset smaller. The highest threshold value is 15, anything below that will decrease the dataset by that percentage. 

In [5]:
def patch_threshold (patch_count,threshold):
    patch_index=[]
    index=0

    for count in patch_count:
        indices=[index+i for i in range(min(threshold,count))]
        index+=count
        patch_index=patch_index+indices
    
    return patch_index

def finalize_files(file_path):
    try:
        os.remove(file_path+"dataset.dat")
    except OSError as e:
        print(f"Error deleting file: {e}")

    try:
        os.rename(file_path+"dataset_temp.dat", file_path+"dataset.dat")
    except OSError as e:
        print(f"Error renaming file: {e}")

def dataset_threshold(file_path,length,threshold):
    dataset=np.memmap(file_path+"dataset.dat", dtype='int8', mode='r', shape=(length,128,128,98))
    patch_count=np.memmap(file_path+"patch.dat", dtype='int8', mode='r', shape=(1775,))

    patch_index=patch_threshold(patch_count,threshold)
    new_len=len(patch_index)

    updated_dataset=np.memmap(file_path+"dataset_temp.dat", dtype='int8', mode='w+', shape=(new_len,128,128,98))

    for pos,index in tqdm(enumerate(patch_index),total=len(patch_index)):
        updated_dataset[pos]=dataset[index]
        updated_dataset.flush()

    dataset.flush()
    patch_count.flush()
    updated_dataset.flush()

    finalize_files(file_path)

    return new_len


dataset.flush()

_dataset_len=dataset_threshold(autofocus_exp_path,_dataset_len,7)

def save_variable_to_file(variable, filename):
    try:
        with open(filename, 'w+') as file:
            file.write(str(variable))
        print(f"Variable saved successfully to {filename}")
    except Exception as e:
        print(f"Error occurred while saving variable to {filename}: {e}")

save_variable_to_file(_dataset_len,autofocus_exp_path+"length.dat")

dataset=np.memmap(autofocus_exp_path+"dataset.dat", dtype='int8', mode='r', shape=(_dataset_len,128,128,98))
dataset.flush()


100%|██████████| 10772/10772 [27:23<00:00,  7.83it/s]


Now we have to make the labels from the dataset using the laplacian distance method.

In [2]:
def apply_laplacian(images):
    max_variance = -1
    focused_index = -1
    
    for i, image in enumerate(images):
        img=(image+128).astype(np.uint8)
        laplacian = cv2.Laplacian(img, cv2.CV_64F)
        variance = laplacian.var()
        
        if variance > max_variance:
            max_variance = variance
            focused_index = i
    
    return focused_index

def parse_image_set(images):
    reshaped_images = np.transpose(images, (2, 0, 1))
    
    focused_index = apply_laplacian(reshaped_images)
    
    return focused_index//2

def predict_ground_truth(file_path,length):

    dataset=np.memmap(file_path+"dataset.dat", dtype='int8', mode='r', shape=(length,128,128,98))
    labels=np.memmap(file_path+"labels.dat", dtype='int8', mode='w+', shape=(length,))

    for i,image_set in tqdm(enumerate(dataset),total=length):
        labels[i]=parse_image_set(image_set)
        labels.flush()

    dataset.flush()
    labels.flush()

predict_ground_truth(autofocus_exp_path,_dataset_len)

dataset=np.memmap(autofocus_exp_path+"dataset.dat", dtype='int8', mode='r', shape=(_dataset_len,128,128,98))
labels=np.memmap(autofocus_exp_path+"labels.dat", dtype='int8', mode='r', shape=(_dataset_len,))


100%|██████████| 10772/10772 [7:36<00:00,  24.94it/s]


Initializing the Test Dataset

In [8]:
autofocus_test_path_cache=autofocus_test_path+"Cache/"

shutil.copy(autofocus_test_path_cache+"dataset.dat",autofocus_exp_path+"Test/dataset.dat")
shutil.copy(autofocus_test_path_cache+"labels.dat",autofocus_exp_path+"Test/labels.dat")

_test_len=int(load_variable_from_file(autofocus_test_path_cache+"length.dat"))

predict_ground_truth(autofocus_exp_path+"Test/",_test_len)

test_data=np.memmap(autofocus_exp_path+"Test/dataset.dat", dtype='int8', mode='r', shape=(_test_len,128,128,98))
test_labels=np.memmap(autofocus_exp_path+"Test/labels.dat", dtype='int8', mode='r', shape=(_test_len,))

Variable loaded successfully from /mnt/Data/Autofocus/Test/Cache/length.dat


100%|██████████| 1251/1251 [00:43<00:00, 28.54it/s]


In [9]:
def load_variable_from_file(filename):
    try:
        with open(filename, 'r') as file:
            variable = file.read()
        print(f"Variable loaded successfully from {filename}")
        return variable
    except Exception as e:
        print(f"Error occurred while loading variable from {filename}: {e}")
        return None
    
_dataset_len=int(load_variable_from_file(autofocus_exp_path+"length.dat"))

dataset=np.memmap(autofocus_exp_path+"dataset.dat", dtype='int8', mode='r', shape=(_dataset_len,128,128,98))
labels=np.memmap(autofocus_exp_path+"labels.dat", dtype='int8', mode='r', shape=(_dataset_len,))


Variable loaded successfully from /mnt/Data/Autofocus/Exp3/length.dat


The MobileNetV2 model is used. The input shape is converted to (128,128,98) to be able to take the dual pixel focal stack slices as individual channels of an image. Then we use ordinal regression loss (L2). The Adam optimizer is used to build the model. 

In [ ]:


# Step 1: Modify MobileNetV2 to accept 128x128x98 input
def create_modified_mobilenetv2(input_shape=(128, 128, 98)):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Assuming the output for the ordinal regression problem is a single value
    outputs = Dense(1)(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Step 2: Implement the ordinal regression loss
def ordinal_regression_loss(y_true, y_pred):
    # L2 loss (mean squared error)
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)

# Step 3: Set up the training loop
def train_model(model, train_dataset, steps_per_epoch=20000, epochs=128, initial_lr=1e-4, beta1=0.5, beta2=0.999):
    optimizer = Adam(learning_rate=initial_lr, beta_1=beta1, beta_2=beta2)
    model.compile(optimizer=optimizer, loss=ordinal_regression_loss,metrics=['accuracy'])
    
    model.fit(train_dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)

# Create the model
input_shape = (128, 128, 98)
model = create_modified_mobilenetv2(input_shape=input_shape)

#model.summary()

The steps_per_epoch is the only thing the user running the model needs to think about. The formula is len(dataset)=batch_size * steps_per_epoch * epochs. So choose a good steps_per_epoch size. Choosing a higher number means consuming more RAM but less time and choosing a lesser number number means less RAM but more time. 

In [11]:
batch_size=128

train_dataset = tf.data.Dataset.from_tensor_slices((dataset, labels)).batch(batch_size)
print("Train Dataset Length :",len(train_dataset))

Train Dataset Length : 84


In [20]:
steps_per_epoch=7
epochs=len(train_dataset)//steps_per_epoch

train_model(model, train_dataset, steps_per_epoch=steps_per_epoch, epochs=epochs)

Epoch 1/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1028ms/step - accuracy: 0.994 - loss: 0.905
Epoch 2/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1080ms/step - accuracy: 0.991 - loss: 1.23
Epoch 3/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1001ms/step - accuracy: 0.972 - loss: 0.689
Epoch 4/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 994ms/step - accuracy: 0.971 - loss: 0.426
Epoch 5/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 984ms/step - accuracy: 1.0 - loss: 0.883
Epoch 6/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1057ms/step - accuracy: 0.99 - loss: 1.202
Epoch 7/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1089ms/step - accuracy: 0.993 - loss: 0.538
Epoch 8/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 960ms/step - accuracy: 0.963 - loss: 0.339
Epoch 9/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 971ms/step - accuracy: 0.967 - loss: 0.919
Epoch 10/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1038ms/step - accuracy: 0.968 - loss: 0.507
Epoch 11/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 1100ms/step - accuracy: 0.996 - loss: 1.026
Epoch 12/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 981ms/step - accuracy: 0.968 - loss: 0.89

In [21]:
model.save(autofocus_exp_path+"Experiment_3_model.keras")

Test the model

In [22]:
batch_size=32

test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(batch_size)
print("Test Dataset Length :",len(test_dataset))

Test Dataset Length : 40


In [18]:
results = model.evaluate(test_dataset)
loss, accuracy = results
print(f"Loss: {loss}, Accuracy: {accuracy*100}%")


40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.982 - loss: 0.017
Loss: 0.017836287493658, Accuracy: 98.274847658239742%


Clear the RAM for immediate clean up

In [3]:
def clear_ram():
    global_vars = list(globals().keys())  # Get a list of global variable names
    vars_to_delete = [var for var in global_vars]
    
    # Delete selected variables
    for var in vars_to_delete:
        del globals()[var]
    import gc
    # Invoke garbage collector
    gc.collect()
    
    # Print confirmation
    print('RAM cleared')

time.sleep(5)

clear_ram()

RAM cleared
